In [4]:
# ==============================================================================
# PROJETO: Marklt BI - Geração de Dados Fictícios (Mock Data)
# AUTOR: Gabriel Lopes Cavallari
# DESCRIÇÃO:
# Script para geração de massa de dados simulada para análise de marketplace.
# Garante integridade referencial entre usuários, anúncios, buscas e vendas.
# ==============================================================================

# 1. INSTALAÇÃO DE DEPENDÊNCIAS
# Instala a biblioteca Faker para gerar dados anonimizados realistas
!pip install faker -q

import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta

# Configuração Inicial
fake = Faker('pt_BR') # Configura para gerar dados no formato brasileiro
random.seed(42) # Seed fixa para garantir reprodutibilidade (os dados serão sempre os mesmos)

print("--- INICIANDO PROCESSO DE GERAÇÃO DE DADOS ---")

# ==============================================================================
# ETAPA 1: DIMENSÃO DE USUÁRIOS (Stakeholders)
# Objetivo: Criar base de clientes respeitando princípios de privacidade (LGPD).
# Nenhum dado real é utilizado.
# ==============================================================================
print("[1/4] Gerando Tabela de Usuários...")

num_users = 500
users = []

for i in range(1, num_users + 1):
    users.append({
        'user_id': i,
        'name': fake.name(),
        'email': fake.free_email(), # E-mails genéricos para evitar domínios corporativos reais
        'join_date': fake.date_between(start_date='-2y', end_date='today')
    })

df_users = pd.DataFrame(users)


# ==============================================================================
# ETAPA 2: TABELA FATO - ANÚNCIOS (Ad Listings)
# Objetivo: Simular o ciclo de vida do anúncio (Postagem -> Venda ou Exclusão).
# Regra de Negócio: A data de venda/exclusão nunca pode ser anterior à data de postagem.
# ==============================================================================
print("[2/4] Gerando Tabela de Anúncios...")

num_ads = 2000
categories = ['Eletrônicos', 'Móveis', 'Roupas', 'Brinquedos', 'Livros', 'Esportes']
ads = []

for i in range(1, num_ads + 1):
    # Data de postagem aleatória no último ano
    posted_date = fake.date_between(start_date='-1y', end_date='today')

    # Define aleatoriamente o status atual do item
    status = random.choice(['Sold', 'Active', 'Deleted'])

    sold_date = None
    deletion_date = None

    # Lógica para Itens Vendidos
    if status == 'Sold':
        days_to_sell = random.randint(1, 60) # Tempo de maturação da venda
        sold_date = posted_date + timedelta(days=days_to_sell)

        # Correção: Se a data de venda projetada for no futuro, ajustamos para hoje
        if sold_date > datetime.now().date():
            sold_date = datetime.now().date()
            posted_date = sold_date - timedelta(days=random.randint(1, 5)) # Ajuste retroativo

    # Lógica para Itens Excluídos (Churn de anúncio)
    elif status == 'Deleted':
        days_active = random.randint(1, 90)
        deletion_date = posted_date + timedelta(days=days_active)

    ads.append({
        'ad_id': i,
        'seller_id': random.randint(1, num_users), # Chave estrangeira para Users
        'category': random.choice(categories),
        'title': f"Item {fake.word().capitalize()} {random.randint(1,100)}", # Nome genérico do produto
        'posted_date': posted_date,
        'sold_date': sold_date,
        'deletion_date': deletion_date,
        'price': round(random.uniform(10, 500), 2), # Preço entre R$10 e R$500
        'status': status
    })

df_ads = pd.DataFrame(ads)


# ==============================================================================
# ETAPA 3: COMPORTAMENTO DE BUSCA (User Searches)
# Objetivo: Analisar intenção de compra vs conversão real.
# Volume alto de dados para simular funil de vendas (muita busca, pouca compra).
# ==============================================================================
print("[3/4] Gerando Histórico de Pesquisas...")

num_searches = 5000
searches = []
# Termos variados para enriquecer a análise textual posterior
search_terms = ['iphone', 'sofá', 'mesa', 'camisa', 'bola', 'ps5', 'monitor', 'livro bi', 'tênis', 'cadeira gamer']

for i in range(1, num_searches + 1):
    searches.append({
        'search_id': i,
        'user_id': random.randint(1, num_users),
        'search_query': random.choice(search_terms),
        'search_date': fake.date_between(start_date='-1y', end_date='today')
    })

df_searches = pd.DataFrame(searches)


# ==============================================================================
# ETAPA 4: TRANSAÇÕES FINANCEIRAS (Sales Transactions)
# Objetivo: Validar receita.
# Regra de Negócio: Somente itens com status 'Sold' geram transações.
# Regra de Negócio: O Comprador (Buyer) não pode ser o mesmo que o Vendedor (Seller).
# ==============================================================================
print("[4/4] Processando Transações Financeiras...")

sales = []
# Filtra apenas anúncios que foram efetivamente vendidos
sold_ads = df_ads[df_ads['status'] == 'Sold']

for index, row in sold_ads.iterrows():
    buyer_id = random.randint(1, num_users)

    # Loop para garantir que ninguém compre o próprio produto (fraude/erro)
    while buyer_id == row['seller_id']:
        buyer_id = random.randint(1, num_users)

    sales.append({
        'transaction_id': f"TXN-{row['ad_id']}-{random.randint(1000,9999)}",
        'ad_id': row['ad_id'],
        'buyer_id': buyer_id,
        'seller_id': row['seller_id'],
        'amount': row['price'],
        'transaction_date': row['sold_date'] # Data da transação = Data da venda
    })

df_sales = pd.DataFrame(sales)

# ==============================================================================
# EXPORTAÇÃO FINAL
# ==============================================================================
print("\n--- SALVANDO ARQUIVOS CSV ---")
df_users.to_csv('marklt_users.csv', index=False)
df_ads.to_csv('marklt_ads.csv', index=False)
df_searches.to_csv('marklt_searches.csv', index=False)
df_sales.to_csv('marklt_sales.csv', index=False)

print("SUCESSO: Arquivos gerados e prontos para download.")
print(f"- Usuários: {len(df_users)} registros")
print(f"- Anúncios: {len(df_ads)} registros")
print(f"- Pesquisas: {len(df_searches)} registros")
print(f"- Vendas Confirmadas: {len(df_sales)} registros")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.4 MB/s eta 0:00:00
--- INICIANDO PROCESSO DE GERAÇÃO DE DADOS ---
[1/4] Gerando Tabela de Usuários...
[2/4] Gerando Tabela de Anúncios...
[3/4] Gerando Histórico de Pesquisas...
[4/4] Processando Transações Financeiras...

--- SALVANDO ARQUIVOS CSV ---
SUCESSO: Arquivos gerados e prontos para download.
- Usuários: 500 registros
- Anúncios: 2000 registros
- Pesquisas: 5000 registros
- Vendas Confirmadas: 640 registros
